In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from sklearn.metrics import classification_report


DATA_ROOT = r"E:\semester 7\DL\tubes_2\data"
TRAIN_DIR = os.path.join(DATA_ROOT, "train")
VAL_DIR = os.path.join(DATA_ROOT, "val")

BATCH_SIZE = 8
NUM_EPOCHS = 25
LR = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


e:\semester 7\DL\tubes_2\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
class FaceDataset(Dataset):
    def __init__(self, root, mtcnn, transform=None):
        self.root = root
        self.mtcnn = mtcnn
        self.transform = transform

        self.samples = []
        self.class_to_idx = {}

        classes = sorted(os.listdir(root))
        for idx, cname in enumerate(classes):
            cpath = os.path.join(root, cname)
            if not os.path.isdir(cpath):
                continue
            self.class_to_idx[cname] = idx

            for f in os.listdir(cpath):
                if f.lower().endswith((".jpg", ".jpeg", ".png")):
                    self.samples.append((os.path.join(cpath, f), idx))

        self.idx_to_class = {v: k for k, v in self.class_to_idx.items()}

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        img = Image.open(path).convert("RGB")

        face = self.mtcnn(img)

        # Jika gagal deteksi, resize saja
        if face is None:
            face = transforms.Resize((160,160))(img)
            face = transforms.ToTensor()(face)

        if self.transform:
            face = self.transform(face)

        return face, label


In [3]:
# MTCNN detection
mtcnn = MTCNN(image_size=160, margin=20, device=device)

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
])

val_transform = None

train_dataset = FaceDataset(TRAIN_DIR, mtcnn, transform=train_transform)
val_dataset   = FaceDataset(VAL_DIR, mtcnn, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False)

num_classes = len(train_dataset.class_to_idx)
num_classes


69

In [4]:
facenet = InceptionResnetV1(pretrained='vggface2').to(device)
facenet.eval()

# Freeze backbone
for p in facenet.parameters():
    p.requires_grad = False

# Cek embedding size
emb_dim = facenet(torch.randn(1,3,160,160).to(device)).shape[1]
print("Embedding dim:", emb_dim)

# Classifier head
classifier = nn.Linear(emb_dim, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=LR)


Embedding dim: 512


In [5]:
os.makedirs("models", exist_ok=True)

In [ ]:
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    facenet.eval()
    classifier.train()

    running_loss = 0
    correct = 0
    total = 0

    # TRAIN
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        with torch.no_grad():
            emb = facenet(imgs)

        outputs = classifier(emb)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

    train_acc = correct / total
    train_loss = running_loss / total

    # VALIDATION
    classifier.eval()
    correct = 0
    total = 0
    running_loss = 0

    all_labels = []
    all_preds  = []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            emb = facenet(imgs)
            outputs = classifier(emb)

            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)

            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    val_acc = correct / total
    val_loss = running_loss / total

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(classifier.state_dict(), "models/facenet_best")

    print(f"[{epoch+1}/{NUM_EPOCHS}] "
          f"Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}")

print("Best Val Acc:", best_val_acc)


[1/25] Train Acc: 0.078 | Val Acc: 0.271
[2/25] Train Acc: 0.420 | Val Acc: 0.529
[3/25] Train Acc: 0.673 | Val Acc: 0.629
[4/25] Train Acc: 0.756 | Val Acc: 0.643
[5/25] Train Acc: 0.780 | Val Acc: 0.686
[6/25] Train Acc: 0.790 | Val Acc: 0.700
[7/25] Train Acc: 0.805 | Val Acc: 0.686
[8/25] Train Acc: 0.810 | Val Acc: 0.700
[9/25] Train Acc: 0.810 | Val Acc: 0.700
[10/25] Train Acc: 0.824 | Val Acc: 0.700
[11/25] Train Acc: 0.800 | Val Acc: 0.700
[12/25] Train Acc: 0.820 | Val Acc: 0.686
[13/25] Train Acc: 0.824 | Val Acc: 0.700
[14/25] Train Acc: 0.805 | Val Acc: 0.700
[15/25] Train Acc: 0.815 | Val Acc: 0.700
[16/25] Train Acc: 0.839 | Val Acc: 0.700
[17/25] Train Acc: 0.824 | Val Acc: 0.714
[18/25] Train Acc: 0.829 | Val Acc: 0.700
[19/25] Train Acc: 0.844 | Val Acc: 0.700
[20/25] Train Acc: 0.849 | Val Acc: 0.729
[21/25] Train Acc: 0.844 | Val Acc: 0.729
[22/25] Train Acc: 0.829 | Val Acc: 0.729
[23/25] Train Acc: 0.844 | Val Acc: 0.729
[24/25] Train Acc: 0.839 | Val Acc: 0.743
[

In [7]:
print(classification_report(
    all_labels, all_preds,
    target_names=[train_dataset.idx_to_class[i] for i in range(num_classes)]
))


                                 precision    recall  f1-score   support

           Abraham Ganda Napitu       0.00      0.00      0.00         1
       Abu Bakar Siddiq Siregar       1.00      1.00      1.00         1
             Ahmad Faqih Hasani       1.00      1.00      1.00         1
                   Aldi Sanjaya       0.00      0.00      0.00         1
                        Alfajar       1.00      1.00      1.00         1
            Alief Fathur Rahman       0.00      0.00      0.00         1
 Arkan Hariz Chandrawinata Liem       1.00      1.00      1.00         1
               Bayu Ega Ferdana       1.00      1.00      1.00         1
          Bayu Prameswara Haris       1.00      1.00      1.00         1
           Bezalel Samuel Manik       1.00      1.00      1.00         1
           Bintang Fikri Fauzan       0.00      0.00      0.00         1
              Boy Sandro Sigiro       0.00      0.00      0.00         1
             Desty Ananta Purba       1.00      1.

e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\metrics\_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\metrics\_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
e:\semester 7\DL\tubes_2\venv\lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The numbe

In [8]:
import json

label_map = train_dataset.class_to_idx  # dari ImageFolder
inv_label_map = {v: k for k, v in label_map.items()}

with open("models/label_map.json", "w") as f:
    json.dump(inv_label_map, f)

print("Label map saved!")


Label map saved!
